## Initial settings

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = f"{7}"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
import tools as t
import joblib

from sklearn.decomposition import PCA
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow import keras
import tensorflow as tf

2024-01-09 18:28:57.001421: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
root_path = "/home/dg321/gitTest/PRI/irp/Ventilation/AAE4Ventilation"

## Load raw data

In [ ]:
# change the data path here
train_1 = pd.read_csv(root_path + "/data_new/6GIC_meeting_room_sensors_2023-11-09.csv")
print(train_1.shape)

(198403, 81)


In [ ]:
print(train_1.head(5))

                  time  time_sin  time_cos  Occupancy  door_gap  window_gap  \
0  2023-06-27 17:53:00 -0.999534 -0.030539        0.0       1.0         2.0   
1  2023-06-27 17:54:00 -0.999657 -0.026177        0.0       1.0         2.0   
2  2023-06-27 17:55:00 -0.999762 -0.021815        0.0       1.0         2.0   
3  2023-06-27 17:56:00 -0.999848 -0.017452        0.0       1.0         2.0   
4  2023-06-27 17:57:00 -0.999914 -0.013090        0.0       1.0         2.0   

   humidity  VOC_ppb  temperature_Main  temperature_FRT  ...  \
0      59.0    157.0             21.99              NaN  ...   
1      59.0    157.0             21.99              NaN  ...   
2      59.0    157.0             21.99              NaN  ...   
3      59.0    157.0             21.99              NaN  ...   
4      59.0    157.0             21.99              NaN  ...   

   outdoor_temperature  outdoor_humidity  outdoor_windgust  outdoor_windspeed  \
0                 15.9              83.2              19.8 

In [ ]:
print(train_1["time"][0])
print(train_1["time"][158721])

2023-06-27 17:53:00
2023-10-04 22:35:00


In [ ]:
train_1['time'] = pd.to_datetime(train_1['time'])

In [ ]:
print(train_1.tail(5))

                      time  time_sin  time_cos  Occupancy  door_gap  \
198398 2023-10-31 19:33:00 -0.918791  0.394744        0.0       0.0   
198399 2023-10-31 19:34:00 -0.917060  0.398749        0.0       0.0   
198400 2023-10-31 19:35:00 -0.915311  0.402747        0.0       0.0   
198401 2023-10-31 19:36:00 -0.913545  0.406737        0.0       0.0   
198402 2023-10-31 19:37:00 -0.911762  0.410719        0.0       0.0   

        window_gap  humidity  VOC_ppb  temperature_Main  temperature_FRT  ...  \
198398    0.000000      49.0    119.0             22.02              NaN  ...   
198399    0.600000      49.0    112.0             22.02              NaN  ...   
198400    0.166667      49.0    112.0             22.02              NaN  ...   
198401    0.000000      49.0    112.0             22.02              NaN  ...   
198402    0.000000      49.0    112.0             22.02              NaN  ...   

        outdoor_temperature  outdoor_humidity  outdoor_windgust  \
198398             

In [ ]:
print(train_1.columns)

Index(['time', 'time_sin', 'time_cos', 'Occupancy', 'door_gap', 'window_gap',
       'humidity', 'VOC_ppb', 'temperature_Main', 'temperature_FRT',
       'temperature_FRM', 'temperature_FRB', 'temperature_FMT',
       'temperature_FMM', 'temperature_FMB', 'temperature_FLT',
       'temperature_FLM', 'temperature_FLB', 'temperature_BRT',
       'temperature_BRM', 'temperature_BRB', 'temperature_BMT',
       'temperature_BMM', 'temperature_BMB', 'temperature_BLT',
       'temperature_BLM', 'temperature_BLB', 'temperature_WRB',
       'temperature_WMB', 'temperature_WLB', 'temperature_WLF',
       'temperature_DoorRT', 'temperature_BTable', 'temperature_PRUR',
       'temperature_PRUL', 'temperature_PRDR', 'temperature_PRDL',
       'temperature_PLDR', 'temperature_PLDL', 'temperature_Out', 'light_FRT',
       'light_FRM', 'light_FRB', 'light_FMT', 'light_FMM', 'light_FMB',
       'light_FLT', 'light_FLM', 'light_FLB', 'light_BRT', 'light_BRM',
       'light_BRB', 'light_BMT', 'light_BMM'

In [ ]:
column_names_list = train_1.columns.tolist()

In [ ]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 198403 entries, 0 to 198402
Data columns (total 81 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   time                      198403 non-null  datetime64[ns]
 1   time_sin                  198403 non-null  float64       
 2   time_cos                  198403 non-null  float64       
 3   Occupancy                 198403 non-null  float64       
 4   door_gap                  198403 non-null  float64       
 5   window_gap                198403 non-null  float64       
 6   humidity                  198403 non-null  float64       
 7   VOC_ppb                   198403 non-null  float64       
 8   temperature_Main          198403 non-null  float64       
 9   temperature_FRT           0 non-null       float64       
 10  temperature_FRM           0 non-null       float64       
 11  temperature_FRB           0 non-null       float64       
 12  te

In [ ]:
# Plot data over the whole time period
# data_after_specific_date = train_1

# # data_after_specific_date = data_after_specific_date.dropna(axis=1)
# print(data_after_specific_date.shape)


# for i in range(data_after_specific_date.shape[1]):
#     plt.figure(figsize=(12, 6))
#     plt.plot(data_after_specific_date['time'], data_after_specific_date.iloc[:,i], label=column_names_list[i])
#     plt.xlabel('Timestamp')
#     plt.ylabel('Your Value')
#     plt.title(column_names_list[i])
#     plt.legend()
#     plt.show()

In [ ]:
specific_date = pd.to_datetime('2023-09-06')
specific_date1 = pd.to_datetime('2023-10-01')

data_after_specific_date = train_1[(train_1['time'] >= specific_date) & (train_1['time'] <= specific_date1)]

data_after_specific_date = data_after_specific_date.dropna(axis=1)
print(data_after_specific_date.shape)


# for i in range(data_after_specific_date.shape[1]):
#     plt.figure(figsize=(12, 6))
#     plt.plot(data_after_specific_date['time'], data_after_specific_date.iloc[:,i], label=column_names_list[i])
#     plt.xlabel('Timestamp')
#     plt.ylabel('Your Value')
#     plt.title(column_names_list[i])
#     plt.legend()
#     plt.show()

(51880, 57)


In [ ]:
print(data_after_specific_date.head(5))
print(data_after_specific_date.shape)

                      time  time_sin  time_cos  Occupancy  door_gap  \
101167 2023-09-06 00:00:00  0.000000  1.000000        0.0       1.0   
101168 2023-09-06 00:01:00  0.004363  0.999990        0.0       1.0   
101169 2023-09-06 00:02:00  0.008727  0.999962        0.0       1.0   
101170 2023-09-06 00:03:00  0.013090  0.999914        0.0       1.0   
101171 2023-09-06 00:04:00  0.017452  0.999848        0.0       1.0   

        window_gap  humidity  VOC_ppb  temperature_Main  temperature_BRT  ...  \
101167         2.0      60.0    195.0             23.68        27.210708  ...   
101168         2.0      60.0    195.0             23.68        27.210708  ...   
101169         2.0      60.0    195.0             23.68        27.210708  ...   
101170         2.0      60.0    195.0             23.68        27.210708  ...   
101171         2.0      60.0    219.0             23.66        27.210708  ...   

        outdoor_temperature  outdoor_humidity  outdoor_windgust  \
101167             

In [ ]:
# Extract features (X)
X = data_after_specific_date.drop(columns=['time']).to_numpy()
print(X.shape)

(51880, 56)


In [ ]:

# Split the data into training, validation, and test sets
# Adjust the test_size and random_state as needed

X_train = X[:36316,:]
# X_val, X_test = train_test_split(X_temp, test_size=0.5, random_state=None)

# # Normalize the features using StandardScaler
# scaler = StandardScaler()

# # Fit on training data and transform the training set
# X_train_normalized = scaler.fit_transform(X_train)

# # Transform the validation and test sets using the same scaler
# X_val_normalized = scaler.transform(X_val)
# X_test_normalized = scaler.transform(X_test)

# print(X_train_normalized.shape)
# print(X_test_normalized.shape)
# print(X_val_normalized.shape)

In [ ]:
fields_list = (data_after_specific_date.drop(columns=['time'])).columns.tolist()

In [ ]:
# np.save(root_path + '/data/X_train_1124.npy', X_train)

In [ ]:
print(X_train.shape)

(36316, 56)


In [ ]:
all_values = X_train

In [ ]:
print(X[:300, 0].shape)

(300,)


## Standardise the Data (could skip this part)

In [ ]:
fields = X_train

In [ ]:
print(fields.shape)

(36316, 56)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# change the output file name
fields_scaled = []

for i in range(0, fields.shape[1]):
    scaler = MinMaxScaler((0, 1))
    # Use fields[:, i] instead of fields[fields_list[i]]
    scaler.fit(fields[:, i].reshape(-1, 1))
    
    # Use fields[:, i] instead of fields[fields_list[i]]
    field_scaled = scaler.transform(fields[:, i].reshape(-1, 1))
    
    joblib.dump(scaler, root_path + '/scalers/1124_train_scaler01_{}.pkl'.format(fields_list[i]))
    
    print(fields[:, i].shape)
    print(field_scaled.shape)
    
    # Append the entire column, not just the reshaped array
    fields_scaled.append(field_scaled)

# Convert the list to a NumPy array
fields_scaled = np.stack(fields_scaled)

(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)
(36316,)
(36316, 1)


In [ ]:
all_values = np.hstack(fields_scaled)

In [ ]:
print(all_values.shape)

(36316, 56)


In [ ]:
np.save(root_path + '/data/all_values_1124.npy', all_values)